In [ ]:
from datetime import datetime #trabalhar com data
import pandas as pd

import pandas_datareader.data as web # p importar dados do yahoo
!pip install yfinance --upgrade --no-cache-dir # para puxar dados do yahoo
import yfinance as yf

import requests #vou precisar enviar uma requisição p um site
from bs4 import BeautifulSoup# webscrep
import plotly#trabalhar com graficos
import openpyxl# p trabalhar com arquivos do excel

Requirement already up-to-date: yfinance in /usr/local/lib/python3.7/dist-packages (0.1.55)


In [ ]:
url = 'https://www.fundsexplorer.com.br/ranking'

In [ ]:
#agent ={"User-Agent":"Mozilla/5.0"} #pode precisar de um agente para coletar os dados
response = requests.get(url) 

sopa = BeautifulSoup(response.text,'lxml')
tabela = sopa.find_all('table')[0]# lendo todas as tabelas da pagina e pegando somente a primeira, acho q so tem uma
df = pd.read_html(str(tabela),decimal=',',thousands='.')[0]
df

,Códigodo fundo,Setor,Preço Atual,Liquidez Diária,Dividendo,DividendYield,DY (3M)Acumulado,DY (6M)Acumulado,DY (12M)Acumulado,DY (3M)Média,DY (6M)Média,DY (12M)Média,DY Ano,Variação Preço,Rentab.Período,Rentab.Acumulada,PatrimônioLíq.,VPA,P/VPA,DYPatrimonial,VariaçãoPatrimonial,Rentab. Patr.no Período,Rentab. Patr.Acumulada,VacânciaFísica,VacânciaFinanceira,QuantidadeAtivos
0,FIVN11,Shoppings,"R$ 3,94",12750.0,"R$ 0,00","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%",NaN,"0,00%","0,00%","0,00%","R$ 67.081.986,41","R$ 7,13",0.55,NaN,NaN,NaN,NaN,"56,00%",NaN,1
1,BZLI11,Títulos e Val. Mob.,"R$ 14,70",440410.0,"R$ 0,00","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","0,68%","0,68%","1,14%","R$ 432.989.541,95","R$ 11,56",1.27,NaN,NaN,NaN,NaN,NaN,NaN,0
2,XTED11,Lajes Corporativas,"R$ 6,97",21540.0,"R$ 0,00","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","1,10%","1,10%","-37,83%","R$ 26.877.323,95","R$ 14,99",0.46,NaN,NaN,NaN,NaN,"100,00%","100,00%",1
3,ALMI11,Lajes Corporativas,"R$ 1.349,80",230.0,"R$ 0,00","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","2,27%","2,27%","-28,29%","R$ 223.465.106,63","R$ 2.009,99",0.67,NaN,NaN,NaN,NaN,"68,15%",NaN,1
4,DOMC11,Lajes Corporativas,"R$ 439,00",420.0,"R$ 0,00","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","4,35%","4,35%","-10,00%","R$ 271.169.512,09","R$ 1.109,62",0.40,NaN,NaN,NaN,NaN,"23,44%",NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,ERCR11,NaN,NaN,NaN,"R$ 630,69","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%",NaN,"0,00%","0,00%","0,00%","R$ 222.673.897,46","R$ 93.717,97",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
207,RBRY11,Títulos e Val. Mob.,"R$ 105,78",82600.0,"R$ 0,75","0,71%","2,33%","4,21%","8,01%","0,78%","0,70%","0,67%","1,42%","-1,40%","-0,69%","0,00%","R$ 208.046.787,64","R$ 106,42",0.99,"0,70%","0,00%","0,70%","0,70%",NaN,NaN,0
208,RBRR11,Títulos e Val. Mob.,"R$ 97,51",218780.0,"R$ 0,62","0,64%","1,90%","3,44%","6,71%","0,63%","0,57%","0,56%","1,26%","0,99%","1,63%","2,27%","R$ 762.442.558,73","R$ 99,82",0.98,"0,60%","0,00%","0,60%","0,60%",NaN,NaN,0
209,RBRF11,Títulos e Val. Mob.,"R$ 92,39",566550.0,"R$ 0,60","0,65%","2,14%","4,47%","7,88%","0,71%","0,74%","0,66%","1,25%","-6,84%","-6,23%","-5,66%","R$ 1.278.126.688,01","R$ 93,43",0.99,"0,64%","0,00%","0,64%","0,64%",NaN,NaN,0


###Tratando os dados


In [ ]:
#pegando os dados q me interresam
df = df.loc[:, ['Códigodo fundo',	'Setor','Preço Atual',	'Liquidez Diária','Dividendo',	'PatrimônioLíq.',	'VPA',	'P/VPA',	'VacânciaFísica',	'VacânciaFinanceira',	'QuantidadeAtivos']]
df

,Códigodo fundo,Setor,Preço Atual,Liquidez Diária,Dividendo,PatrimônioLíq.,VPA,P/VPA,VacânciaFísica,VacânciaFinanceira,QuantidadeAtivos
0,FIVN11,Shoppings,"R$ 3,94",12750.0,"R$ 0,00","R$ 67.081.986,41","R$ 7,13",0.55,"56,00%",NaN,1
1,BZLI11,Títulos e Val. Mob.,"R$ 14,70",440410.0,"R$ 0,00","R$ 432.989.541,95","R$ 11,56",1.27,NaN,NaN,0
2,XTED11,Lajes Corporativas,"R$ 6,97",21540.0,"R$ 0,00","R$ 26.877.323,95","R$ 14,99",0.46,"100,00%","100,00%",1
3,ALMI11,Lajes Corporativas,"R$ 1.349,80",230.0,"R$ 0,00","R$ 223.465.106,63","R$ 2.009,99",0.67,"68,15%",NaN,1
4,DOMC11,Lajes Corporativas,"R$ 439,00",420.0,"R$ 0,00","R$ 271.169.512,09","R$ 1.109,62",0.40,"23,44%",NaN,1
...,...,...,...,...,...,...,...,...,...,...,...
206,ERCR11,NaN,NaN,NaN,"R$ 630,69","R$ 222.673.897,46","R$ 93.717,97",NaN,NaN,NaN,0
207,RBRY11,Títulos e Val. Mob.,"R$ 105,78",82600.0,"R$ 0,75","R$ 208.046.787,64","R$ 106,42",0.99,NaN,NaN,0
208,RBRR11,Títulos e Val. Mob.,"R$ 97,51",218780.0,"R$ 0,62","R$ 762.442.558,73","R$ 99,82",0.98,NaN,NaN,0
209,RBRF11,Títulos e Val. Mob.,"R$ 92,39",566550.0,"R$ 0,60","R$ 1.278.126.688,01","R$ 93,43",0.99,NaN,NaN,0


In [ ]:
# Vacancia fisica = area vaga/ area total 
# Vacancia financeira = preco do aluguel da area vaga/ valor de tudo alugado
# Vpa = valor patrimonial/qtd de acoes
#liquidez diaria = qtd de negocios
df['Códigodo fundo']=df['Códigodo fundo'].astype('string')
df['Setor']=df['Setor'].astype('string')


for coluna in ['Preço Atual','Dividendo',	'Liquidez Diária',	'PatrimônioLíq.',	'VPA']:
  df[coluna]=df[coluna].astype('string')
  df[coluna]=df[coluna].str.replace('R','')
  df[coluna]=df[coluna].str.replace('$','')
  df[coluna]=df[coluna].str.replace('.','')
  df[coluna]=df[coluna].str.replace(',','.')
  df[coluna]=pd.to_numeric(df[coluna], errors='coerce')

df['Liquidez Diária']=df['Liquidez Diária']/10 #pois so ai o fundsExplorer colocou ponto como virgula
#P/VPA e QuantidadeAtivos ja estao certos

for coluna in ['VacânciaFísica',	'VacânciaFinanceira']:
  df[coluna]=df[coluna].astype('string')
  df[coluna]=df[coluna].str.replace('%','')
  df[coluna]=df[coluna].str.replace(',','.')
  df[coluna]=pd.to_numeric(df[coluna], errors='coerce')
  df[coluna]=df[coluna]/100

df = df[df['Liquidez Diária'] > 10] # tirando empresas q negociam menos 
df = df[df['Dividendo'] != 0] # tirando empresas q negociam menos 

df  

,Códigodo fundo,Setor,Preço Atual,Liquidez Diária,Dividendo,PatrimônioLíq.,VPA,P/VPA,VacânciaFísica,VacânciaFinanceira,QuantidadeAtivos
7,KNRE11,Residencial,3.29,35030.0,0.26,5.443446e+07,2.93,1.12,NaN,NaN,0
8,FMOF11,Lajes Corporativas,70.05,1390.0,0.27,6.718540e+07,132.25,0.53,0.4792,NaN,1
9,ATSA11,Shoppings,113.98,90.0,0.25,1.732830e+08,98.95,1.15,0.1850,NaN,1
10,XPHT11,Hotel,85.51,110.0,0.34,4.746073e+08,124.74,0.69,0.7770,NaN,5
12,BPML11,Shoppings,96.76,19030.0,0.28,5.930961e+08,137.77,0.70,0.0580,NaN,7
...,...,...,...,...,...,...,...,...,...,...,...
205,IRDM11,Títulos e Val. Mob.,137.00,1290720.0,1.23,1.596556e+09,99.78,1.37,NaN,NaN,0
207,RBRY11,Títulos e Val. Mob.,105.78,82600.0,0.75,2.080468e+08,106.42,0.99,NaN,NaN,0
208,RBRR11,Títulos e Val. Mob.,97.51,218780.0,0.62,7.624426e+08,99.82,0.98,NaN,NaN,0
209,RBRF11,Títulos e Val. Mob.,92.39,566550.0,0.60,1.278127e+09,93.43,0.99,NaN,NaN,0


In [ ]:
df.dtypes #sabendo os tipos das variaveis

Códigodo fundo         string
Setor                  string
Preço Atual           float64
Liquidez Diária       float64
Dividendo             float64
PatrimônioLíq.        float64
VPA                   float64
P/VPA                 float64
VacânciaFísica        float64
VacânciaFinanceira    float64
QuantidadeAtivos        int64
dtype: object

##Pegando os Alugueis


In [ ]:
#funcao q vai puxar os dividendos de cada fundo

#agent ={"User-Agent":"Mozilla/5.0"} #pode precisar de um agente para coletar os dados
def dividendosFundo(fundo):
  try:
    url = 'https://fiis.com.br/{}/'.format(fundo)
    response = requests.get(url) 
    sopa = BeautifulSoup(response.text,'lxml')
    tabela = sopa.find_all('table')[0]# lendo todas as tabelas da pagina e pegando somente a primeira, acho q so tem uma
    df = pd.read_html(str(tabela),decimal=',',thousands='.')[0]
    # ---------------tratando----------------------
    
    df['Rendimento']=df['Rendimento'].astype('string')
    df['Rendimento']=df['Rendimento'].str.replace('R','')
    df['Rendimento']=df['Rendimento'].str.replace('$','')
    df['Rendimento']=df['Rendimento'].str.replace('.','')
    df['Rendimento']=df['Rendimento'].str.replace(',','.')
    df['Rendimento']=pd.to_numeric(df['Rendimento'], errors='coerce')
    
    return  df.Rendimento.sum()/10, df.Rendimento.median() #retorna media e mediana
  except:
    #print('NAO DEU CERTO PARA O TICK', fundo)
    return  0, 0


In [ ]:
df['Dividendo_Medio'] =0
df['Dividendo_Mediana']=0
df

,Códigodo fundo,Setor,Preço Atual,Liquidez Diária,Dividendo,PatrimônioLíq.,VPA,P/VPA,VacânciaFísica,VacânciaFinanceira,QuantidadeAtivos,Dividendo_Medio,Dividendo_Mediana
7,KNRE11,Residencial,3.29,35030.0,0.26,5.443446e+07,2.93,1.12,NaN,NaN,0,0,0
8,FMOF11,Lajes Corporativas,70.05,1390.0,0.27,6.718540e+07,132.25,0.53,0.4792,NaN,1,0,0
9,ATSA11,Shoppings,113.98,90.0,0.25,1.732830e+08,98.95,1.15,0.1850,NaN,1,0,0
10,XPHT11,Hotel,85.51,110.0,0.34,4.746073e+08,124.74,0.69,0.7770,NaN,5,0,0
12,BPML11,Shoppings,96.76,19030.0,0.28,5.930961e+08,137.77,0.70,0.0580,NaN,7,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,IRDM11,Títulos e Val. Mob.,137.00,1290720.0,1.23,1.596556e+09,99.78,1.37,NaN,NaN,0,0,0
207,RBRY11,Títulos e Val. Mob.,105.78,82600.0,0.75,2.080468e+08,106.42,0.99,NaN,NaN,0,0,0
208,RBRR11,Títulos e Val. Mob.,97.51,218780.0,0.62,7.624426e+08,99.82,0.98,NaN,NaN,0,0,0
209,RBRF11,Títulos e Val. Mob.,92.39,566550.0,0.60,1.278127e+09,93.43,0.99,NaN,NaN,0,0,0


In [ ]:
todos_cod=df.loc[:,'Códigodo fundo']
i=0
for cod in todos_cod:
  df.Dividendo_Medio.iloc[i], df.Dividendo_Mediana.iloc[i]  =  dividendosFundo(cod)
  i=i+1
df

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning:


A v

In [ ]:
selic = 0.03
premio_de_risco =0.03
taxa_de_desconto=selic + premio_de_risco 


taxa_de_desconto = ((taxa_de_desconto+1)**(1/12))-1 #passando a taxa de desconto p mensal
taxa_de_desconto

In [ ]:
df['Valuation_Medio'] =0
df['Valuation_Mediana']=0

todos_cod=df.loc[:,'Códigodo fundo']
i=0
for cod in todos_cod:
  df.Valuation_Medio.iloc[i] = df.Dividendo_Medio.iloc[i]/taxa_de_desconto
  df.Valuation_Mediana.iloc[i]  =  df.Dividendo_Mediana.iloc[i]/taxa_de_desconto
  i=i+1
df

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Códigodo fundo,Setor,Preço Atual,Liquidez Diária,Dividendo,PatrimônioLíq.,VPA,P/VPA,VacânciaFísica,VacânciaFinanceira,QuantidadeAtivos,Dividendo_Medio,Dividendo_Mediana,Valuation_Medio,Valuation_Mediana
7,KNRE11,Residencial,3.29,35030.0,0.26,5.443446e+07,2.93,1.12,NaN,NaN,0,0.097,0.095,19.927887,19.517003
8,FMOF11,Lajes Corporativas,70.05,1390.0,0.27,6.718540e+07,132.25,0.53,0.4792,NaN,1,0.384,0.270,78.889781,55.469378
9,ATSA11,Shoppings,113.98,90.0,0.25,1.732830e+08,98.95,1.15,0.1850,NaN,1,0.222,0.250,45.608155,51.360535
10,XPHT11,Hotel,85.51,110.0,0.34,4.746073e+08,124.74,0.69,0.7770,NaN,5,0.426,0.425,87.518351,87.312909
12,BPML11,Shoppings,96.76,19030.0,0.28,5.930961e+08,137.77,0.70,0.0580,NaN,7,0.251,0.440,51.565977,90.394541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,IRDM11,Títulos e Val. Mob.,137.00,1290720.0,1.23,1.596556e+09,99.78,1.37,NaN,NaN,0,1.152,1.190,236.669344,244.476145
207,RBRY11,Títulos e Val. Mob.,105.78,82600.0,0.75,2.080468e+08,106.42,0.99,NaN,NaN,0,0.692,0.650,142.165960,133.537390
208,RBRR11,Títulos e Val. Mob.,97.51,218780.0,0.62,7.624426e+08,99.82,0.98,NaN,NaN,0,0.542,0.525,111.349639,107.857123
209,RBRF11,Títulos e Val. Mob.,92.39,566550.0,0.60,1.278127e+09,93.43,0.99,NaN,NaN,0,0.654,0.600,134.359159,123.265283


In [ ]:
from google.colab import files
df.to_excel('df.xlsx')
files.download('df.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>